In [1]:
%load_ext autoreload

# Canonical form

In [2]:
import numpy as np
from mps.state import *

## Environment 

A matrix product state form facilitates a decomposition of a complex quantum state in terms of a given site and a left and right environment, formed by the rest of the quantum subsystems.

<img src="figures/environment.svg" style="max-width:90%; width: 22em">

Mathematically, we are performing a Schmidt-type decomposition of the state
$$|\psi\rangle = \sum_{\alpha,i_3,\beta} C^{i_3}_{\alpha\beta}|L^\alpha\rangle|i_3\rangle|R^\beta\rangle$$
with some states $|L^\alpha\rangle$ and $|R^\beta\rangle$ that define a many-body basis for the left and right environments of our central subsystem.

## Canonical form

We claim that the MPS is in *canonical form* with respect to the site $i$ when its left and right tensors define orthonormal basis for their many-body states. In other words, when
$$\langle L^\alpha | L^{\alpha'}\rangle = \delta_{\alpha,\alpha'}$$
$$\langle R^\beta | R^{\beta'}\rangle = \delta_{\beta,\beta'}$$


We can achieve a canonical form by imposing that the tensors to the left and to the right of our subsystem be isometries. In our particular example
$$\sum_i A^{i}_{1,\alpha} A^{i *}_{1,\alpha'} = \delta_{\alpha,\alpha'}$$
$$\sum_{i,\alpha} B^{i}_{\alpha,\beta} B^{i *}_{\alpha,\beta'} = \delta_{\beta,\beta'}$$
$$\sum_{i,\beta} D^{i}_{\alpha,\beta} D^{i *}_{\alpha',\beta} = \delta_{\alpha,\alpha'}$$
$$\sum_i E^{i}_{\alpha,1}E^{i *}_{\alpha',1} = \delta_{\alpha,\alpha'}$$

Or graphically, we can summarize these equations as follows

<img src="figures/canonical-conditions.svg" style="max-width:95%; width:60em">

## Advantages

There are various places where a canonical form becomes very useful. One is when we want to take expectation values of observables. Suppose we wish to compute the average of an observable acting on the third site above
$$\bar{O} = \langle \psi |1 \otimes 1 \otimes O \otimes 1 \otimes 1 |\psi\rangle.$$
If the state is in canonical form, the expectation value can be obtained as a contraction over the local tensors
$$\bar{O} = \sum_{ij\alpha\beta} O_{ij} C^{j*}_{\alpha\beta} C^{i}_{\alpha\beta}$$

Once more, there is a graphical way to express this relation:

<img src="figures/local-expectation-value.svg" style="width: 8em">

The locality of this relation is particularly useful when optimizing expectation values: we can tune the affected tensors independently, until the optimal condition is reached.

## Canonicalizing a tensor

From the images above we guess that there are different canonical conditions depending on whether we come from the left or the right of a given site. If we come from the left, we can start with a tensor that does not satisfy a canonical form and construct a new one that does.

<img src="figures/canonical-split-right.svg" style="max-width: 90%; width: 35em">

Take for instance the tensor $B^{i}_{\alpha\beta},$ which does not satisfy a canonical form. What we do is to reinterpret $B$ as a matrix $C_{x,\beta}$, where the index $x=[\alpha,i_2]$ is born out of joining two legs of the tensor. This matrix admits a singular value decomposition (SVD)
$$C_{x\beta} = \sum_\gamma U_{x,\gamma} s_\gamma V_{\gamma,\beta}$$
with two unitary matrices $U, V$ and a diagonal matrix of non-negative values $s_\gamma.$

We define the tensor
$$\bar{B}^{i_2}_{\alpha\gamma} := U_{x,\gamma}$$
as our new tensor for the second site. The remaining transformations $s V$ are shifted to the next site and, in this particular case, used to update the $C$ tensor to a new tensor
$$\bar{C}_{\gamma\delta}^{i_3} = s_\gamma V_{\gamma\sigma} C_{\sigma\delta}^{i_3}.$$

We can implement this idea as a generic algorithm that updates an MPS, assuming that it is in canonical form up to site $i\pm1$ and moving to site $i$. The algorithm takes the MPS, a generic tensor, the site to update and the direction along which we are moving.

The first part of the algorithm is the splitting of the tensors. We create two functions for this task, `ortho_right()` and `ortho_left()` depending on the direction.

In [3]:
# file: mps/state.py


def _ortho_right(A, tol):
    α, i, β = A.shape
    U, s, V = np.linalg.svd(np.reshape(A, (α*i, β)), full_matrices=False)
    s = _truncate_vector(s, tol)
    D = s.size
    return np.reshape(U, (α, i, D)), np.reshape(s, (D, 1)) * V[:D, :]


def _ortho_left(A, tol):
    α, i, β = A.shape
    U, s, V = np.linalg.svd(np.reshape(A, (α, i*β)), full_matrices=False)
    s = _truncate_vector(s, tol)
    D = s.size
    return np.reshape(V, (D, i, β)), U[:, :D] * np.reshape(s, (1, D))

With the functions above we can now construct the actual update of the MPS at a given site.

In [4]:
# file: mps/state.py


def _update_in_canonical_form(Ψ, A, site, direction, tol=0):
    """Insert a tensor in canonical form into the MPS Ψ at the given site.
    Update the neighboring sites in the process."""

    if direction > 0:
        if site+1 == Ψ.size:
            Ψ[site] = A
        else:
            Ψ[site], sV = _ortho_right(A, tol)
            site += 1
            Ψ[site] = np.einsum('ab,bic->aic', sV, Ψ[site])
    else:
        if site == 0:
            Ψ[site] = A
        else:
            Ψ[site], Us = _ortho_left(A, tol)
            site -= 1
            Ψ[site] = np.einsum('aib,bc->aic', Ψ[site], Us)
    return site

This algorithm can be used iteratively to make an MPS into canonical form with respect to a given site, even if it was not previously so.

In [5]:
# file: mps/state.py

def _canonicalize(Ψ, center):
    for i in range(0, center):
        _update_in_canonical_form(Ψ, Ψ[i], i, +1)
    for i in range(Ψ.size-1, center, -1):
        _update_in_canonical_form(Ψ, Ψ[i], i, -1)

Implicit in the canonicalization is a routine that truncates the singular values up to a tolerance, ensuring that the MPS we regenerate is not too large. We can use the norm-2 criterion for that. The idea is that the sum of the squares of the singular values is the norm of the full state
$$\|\psi\|^2 = \sum_n s_n^2 =: N$$
If we drop all values from $n_{cut}$ on, we make a norm-2 relative error
$$\varepsilon = \frac{1}{N} \sum_{n=n_0} s_n^2$$
We can study the relative error and use it to control our tolerance.

In [6]:
# file: mps/state.py


def _truncate_vector(S, tolerance):
    #
    # Input:
    # - S: a vector containing singular values in descending order
    # - tolerance: truncation relative tolerance, which specifies an
    #   upper bound for the sum of the squares of the singular values
    #   eliminated. 0 <= tolerance <= 1
    #
    # Output:
    # - truncS: truncated version of S
    #
    if tolerance == 0:
        #log('--no truncation')
        return S
    # We sum all reduced density matrix eigenvalues, starting from
    # the smallest ones, to avoid rounding errors
    err = np.cumsum(np.flip(S, axis=0)**2)
    #
    # This is the sum of all values
    total = err[-1]
    #
    # we find the number of values we can drop within the relative
    # tolerance
    ndx = np.argmax(err >= tolerance*total)
    # and use that to estimate the size of the array
    # log('--S='+str(S))
    #log('--truncated to '+str(ndx))
    return S[0:(S.size - ndx)]

## Canonical form MPS

We can use this idea to implement an MPS class that is in canonical form with respect to one site. This site may change as we update the MPS, but it is always kept track of.

In [7]:
# file: mps/state.py


class CanonicalMPS(MPS):
    """Canonical MPS class.

    This implements a Matrix Product State object with open boundary
    conditions, that is always on canonical form with respect to a given site.
    The tensors have three indices, A[α,i,β], where 'α,β' are the internal
    labels and 'i' is the physical state of the given site.

    Attributes:
    size = number of tensors in the array
    center = site that defines the canonical form of the MPS
    """

    #
    # This class contains all the matrices and vectors that form
    # a Matrix-Product State.
    #
    def __init__(self, data, center=0):
        super(MPS, self).__init__(data)
        _canonicalize(self, center)
        self.center = center

    def norm2(self):
        """Return the square of the norm-2 of this state, ‖ψ‖**2 = <ψ|ψ>."""
        A = self._data[self.center]
        return np.vdot(A, A)

    def expectationAtCenter(self, operator):
        """Return the expectation value of 'operator' acting on the central
        site of the MPS."""
        A = self._data[self.center]
        return np.vdot(A, np.einsum('ij,ajb->aib', operator, A))/np.vdot(A,A)

    def update_canonical(self, A, direction):
        self.center = _update_in_canonical_form(self, A, self.center,
                                                direction)

----


# Tests

To properly test the canonical forms we have to verify that the tensors are close to isometries. The following function is a helper for that.

In [8]:
# file: mps/test/test_canonical.py
import unittest
import mps.state
from mps.tools import similar


def approximateIsometry(A, direction, places=7):
    if direction > 0:
        a, i, b = A.shape
        A = np.reshape(A, (a*i, b))
        C = A.T.conj() @ A
    else:
        b, i, a = A.shape
        A = np.reshape(A, (b, i*a))
        C = A @ A.T.conj()
    return np.all(np.isclose(C, np.eye(b), atol=10**(-places)))

In [9]:
# file: mps/test/test_canonical.py


class TestCanonicalForm(unittest.TestCase):

    def test_local_update_canonical(self):
        #
        # We verify that _update_in_canonical_form() leaves a tensor that
        # is an approximate isometry.
        #
        for nqubits in range(2, 10):
            for _ in range(20):
                Ψ = mps.state.random(2, nqubits, 10)

                for i in range(Ψ.size-1):
                    ξ = Ψ.copy()
                    _update_in_canonical_form(ξ, ξ[i], i, +1)
                    self.assertTrue(approximateIsometry(ξ[i], +1))

                for i in range(1, Ψ.size):
                    ξ = Ψ.copy()
                    _update_in_canonical_form(ξ, ξ[i], i, -1)
                    self.assertTrue(approximateIsometry(ξ[i], -1))

    def test_canonicalize(self):
        #
        # We verify _canonicalize() transforms an MPS into an equivalent one
        # that is in canonical form and represents the same state, up to
        # a reasonable tolerance.
        #
        for nqubits in range(2, 10):
            for _ in range(20):
                Ψ = mps.state.random(2, nqubits, 10)

                for center in range(Ψ.size):
                    ξ = Ψ.copy()
                    _canonicalize(ξ, center)
                    #
                    # All sites to the left and to the right are isometries
                    #
                    for i in range(center):
                        self.assertTrue(approximateIsometry(ξ[i], +1))
                    for i in range(center+1, ξ.size):
                        self.assertTrue(approximateIsometry(ξ[i], -1))
                    #
                    # Both states produce the same wavefunction
                    #
                    self.assertTrue(similar(ξ.tovector(), Ψ.tovector()))

    def test_canonical_mps(self):
        #
        # We verify _canonicalize() transforms an MPS into an equivalent one
        # that is in canonical form and represents the same state, up to
        # a reasonable tolerance.
        #
        for nqubits in range(2, 8):
            for _ in range(20):
                Ψ = mps.state.random(2, nqubits, 10)

                for center in range(Ψ.size):
                    ξ = CanonicalMPS(Ψ, center=center)
                    #
                    # All sites to the left and to the right are isometries
                    #
                    for i in range(center):
                        self.assertTrue(approximateIsometry(ξ[i], +1))
                    for i in range(center+1, ξ.size):
                        self.assertTrue(approximateIsometry(ξ[i], -1))
                    #
                    # Both states produce the same wavefunction
                    #
                    self.assertTrue(similar(ξ.tovector(), Ψ.tovector()))
                    #
                    # The norm is correct
                    #
                    self.assertAlmostEqual(ξ.norm2(), Ψ.norm2())
                    #
                    # Local observables give the same
                    #
                    O = np.array([[0, 0],[0, 1]])
                    
                    self.assertAlmostEqual(ξ.expectationAtCenter(O),
                                           Ψ.expectation1(O, center))

In [10]:
%autoreload
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestCanonicalForm'])
unittest.TextTestRunner(verbosity=2).run(suite1);

test_canonical_mps (__main__.TestCanonicalForm) ... ok
test_canonicalize (__main__.TestCanonicalForm) ... ok
test_local_update_canonical (__main__.TestCanonicalForm) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.920s

OK
